# Notebook for: Comparing cities

# Objectives

Find the best place to locate a gaming company.

**Scheme of the company:**

- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President.


**Requirements:**  

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Account managers need to travel a lot.
- Everyone in the company is between 25 and 40, give them some place to go party.
- The CEO is vegan.
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

# Libraries

In [1]:
import os
import pandas as pd
import time
import re
import io
import requests
import json
from dotenv import load_dotenv
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

# Foursquare API requests

I will be requesting information to **Foursquare** using the coordinates of the company that located at the center of the respective gaming-design-tech hub:
- San Francisco: Globant **(37.781929, -122.404176)** / Meez (37.785271, -122.397582)
- South San Francisco: NGM Biopharmaceuticals (37.656246, -122.399735)
- New York: Cellufun (40.739930, -73.993049)
- London (GBR): paymo (51.514165, -0.109017)

**Loading token**

In [2]:
load_dotenv()

True

In [3]:
token_fsq = os.getenv("token_foursquare")

**Function to request from Foursquare**

In [4]:
def getFoursquare(query, location, radius, limit=1):
    '''This function requests information from Foursquare Place Search:
    :query: str, your query in Foursquare.
    :location: list, in the format: [latitude, longitude], both int.
    :radius: int, to limit the radius of the request.
    :limit: int, to limit the number of requests. By default 1.
    '''
        
    ll = f"{location[0]}%2C{location[1]}"
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&radius={radius}&sort=DISTANCE&limit={str(limit)}"

    headers = {
        "accept": "application/json",
        "Authorization": token_fsq,
    }
    
    response = requests.get(url, headers=headers).json()
    
    return response

**Function to request from Foursquare specifying the categories**

In [5]:
def getFoursquareCategory(query, location, radius, categories, limit=1):
    '''This function requests information from Foursquare Place Search:
    :query: str, your query in Foursquare.
    :location: list, in the format: [latitude, longitude], both int.
    :radius: int, to limit the radius of the request.
    :categories: str, the category or categories ID, separated by commas.
    :limit: int, to limit the number of requests. By default 1.
    '''
        
    ll = f"{location[0]}%2C{location[1]}"
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&radius={radius}&categories={categories}&sort=DISTANCE&limit={str(limit)}"

    headers = {
        "accept": "application/json",
        "Authorization": token_fsq,
    }
    
    response = requests.get(url, headers=headers).json()
    
    return response

**Function to extract the information from the Foursquare response**

In [6]:
def getInfo(response, query):
    '''This function receives a request response from foursquare and the info you queried, 
    and returns a dataframe with the following information (columns):
    name, distance, address, latitude, longitude and geometry.
    :response: list, the response of the request from foursquare.
    :query: string, the name of your query in foursquare.
    '''
    
    geo_list = []
    
    for i in response["results"]:

        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]
        distance = i["distance"]

        type_geo = {"typepoint": 
                              {"type": "Point", 
                               "coordinates": [lat, lon]}}

        geo_list.append({"query": query,"name":name, "address":address, "distance":distance, "lat":lat, "lon":lon, "type":type_geo})
    
    df = pd.DataFrame(geo_list)
    
    return df

**Function to get every information from Foursquare and return a concatenated df**

In [7]:
def getEverythingFoursquare(city):
    '''This functions receives the coordinates of a city and returns a dataframe with the extracted information.
    :city: list, in the format [latitude, longitude] both integers.
    '''
    
    # Run all functions
    schools = getFoursquareCategory("school", city, 2000, 12058, limit=5)
    clubs = getFoursquareCategory("night club", city, 2000, 10032, limit=5)
    starbucks = getFoursquare("starbucks", city, 2000, limit=5)
    basketball = getFoursquare("basketball stadium", city, 10000, limit=5)
    dog = getFoursquareCategory("dog grooming", city, 10000, 11134, limit=5)
    airport = getFoursquareCategory("airport", city, 50000, 19040, limit=5)
    vegan = getFoursquareCategory("vegan restaurant", city, 2000, 13377, limit=5)
    
    # Transform to dataframes
    df_1 = getInfo(schools, "school")
    df_2 = getInfo(clubs, "club")
    df_3 = getInfo(starbucks, "starbucks")
    df_4 = getInfo(airport, "airport")
    df_5 = getInfo(vegan, "vegan restaurant")
    df_6 = getInfo(basketball, "basketball")
    df_7 = getInfo(dog, "dog hairdresser")
    
    # Concatenate dataframes
    df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7], ignore_index = True, axis = 0)
    
    return df

**Function to transform typepoint coordinates to geometry**

In [8]:
def geoDataframe(df):
    '''This functions converts the typepoint coordinates of a column of a dataframe so it can be used with CartoFrames.
    '''
    df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]))
    df = df.drop(columns = 'type', axis = 1)
    return df

**Defining coordinates based on other companies' location**

In [9]:
sanfran = [37.781929, -122.404176]
south_sanfran = [37.656246, -122.399735]
newyork = [40.739930, -73.993049]
london = [51.514165, -0.109017]

## San Francisco

In [10]:
# sanfran_df = getEverythingFoursquare(sanfran)

In [26]:
sanfran_df.head()

,query,name,address,distance,lat,lon,type,geometry
0,school,AltSchool Yerba Buena,"300 4th St, San Francisco, CA 94107",255,37.781815,-122.401127,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40113 37.78182)
1,school,San Francisco Unified School District,"825 Shotwell St, San Francisco, CA 94110",348,37.780650,-122.400556,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40056 37.78065)
2,school,American College of Early Childhood Education,"760 Market St, San Francisco, CA 94102",503,37.786362,-122.405338,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40534 37.78636)
3,school,Bessie Carmichael Elementary School,"55 Sherman St, San Francisco, CA 94103",561,37.777146,-122.406228,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40623 37.77715)
4,school,Bessie Carmichael Pre-K School,"55 Sherman St, San Francisco, CA 94103",612,37.776734,-122.406491,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40649 37.77673)


In [12]:
sanfran_geodf = geoDataframe(sanfran_df)

In [13]:
sanfran_geodf.sample()

,query,name,address,distance,lat,lon,geometry
17,vegan restaurant,Mixt,"51 Yerba Buena Ln, San Francisco, CA 94103",424,37.785581,-122.404432,POINT (-122.40443 37.78558)


**Exporting dataset**

In [32]:
sanfran_df.to_csv('./data/sanfrancisco_activities.csv', index = False)
sanfran_geodf.to_csv('./data/sanfrancisco_geo_activities.csv', index = False)

## South San Francisco

In [14]:
# south_sanfran_df = getEverythingFoursquare(south_sanfran)

In [27]:
south_sanfran_df.head()

,query,name,address,distance,lat,lon,type,geometry
0,school,Martin Elementary School,"35 School St, South San Francisco, CA 94080",1195,37.663471,-122.410062,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.41006 37.66347)
1,school,Academic Insight,"325 Corey Way, South San Francisco, CA 94080",1259,37.644996,-122.401457,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40146 37.64500)
2,school,All Souls Catholic School,"479 Miller Ave (Spruce), South San Francisco, ...",1369,37.657254,-122.415280,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.41528 37.65725)
3,school,Spruce Elementary School,"501 Spruce Ave, South San Francisco, CA 94080",1408,37.658973,-122.416312,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.41631 37.65897)
4,school,South San Francisco Unified School District,"530 Tamarack Ln, South San Francisco, CA 94080",1631,37.658713,-122.418015,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.41801 37.65871)


In [16]:
south_sanfran_geodf = geoDataframe(south_sanfran_df)

In [17]:
south_sanfran_geodf.sample()

,query,name,address,distance,lat,lon,geometry
6,club,Z.O.Y. Zumba Fitness Studio,"415 Grand Ave, South San Francisco, CA 94080",1178,37.65562,-122.413108,POINT (-122.41311 37.65562)


**Exporting dataset**

In [33]:
south_sanfran_df.to_csv('./data/south_sanfrancisco_activities.csv', index = False)
south_sanfran_geodf.to_csv('./data/south_sanfrancisco_geo_activities.csv', index = False)

## New York

In [28]:
# newyork_df = getEverythingFoursquare(newyork)

In [29]:
newyork_df.head()

,query,name,address,distance,lat,lon,type,geometry
0,school,Holocaust Education Foundation,"20 W 20th St, New York, NY 10011",52,40.740084,-73.992460,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-73.99246 40.74008)
1,school,Victory Schools,"18 W 18th St, New York, NY 10011",118,40.738875,-73.993286,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-73.99329 40.73888)
2,school,Ps 340,"590 Avenue of the Americas (17th St.), New Yor...",203,40.739036,-73.995301,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-73.99530 40.73904)
3,school,Kane School Of Core Integration,"7 E 17th St, New York, NY 10003",256,40.737837,-73.991761,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-73.99176 40.73784)
4,school,Shield Institute,"114 W 17th St, New York, NY 10011",277,40.739441,-73.996278,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-73.99628 40.73944)


In [20]:
newyork_geodf = geoDataframe(newyork_df)

In [21]:
newyork_geodf.sample()

,query,name,address,distance,lat,lon,geometry
31,dog hairdresser,People + Pets,"218 E 89th St (between 2nd and 3rd), New York,...",5664,40.780349,-73.952,POINT (-73.95200 40.78035)


**Exporting dataset**

In [34]:
newyork_df.to_csv('./data/newyork_activities.csv', index = False)
newyork_geodf.to_csv('./data/newyork_geo_activities.csv', index = False)

## London

In [30]:
# london_df = getEverythingFoursquare(london)

In [31]:
london_df.head()

,query,name,address,distance,lat,lon,type,geometry
0,school,St Albans Primary School,"Baldwins Gardens, London, EC1N 7SD",681,51.520223,-0.110579,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-0.11058 51.52022)
1,school,St Clement Danes C of E Primary School,"Drury Lane, London, Greater London, WC2B 5SU",732,51.513546,-0.119311,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-0.11931 51.51355)
2,school,St Josephs R C Primary School,"Macklin St, London, WC2B 5NA",921,51.516073,-0.121987,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-0.12199 51.51607)
3,school,Christopher Hatton Primary School,"38 Laystall St, London, Greater London, EC1R 4PQ",985,51.522687,-0.112286,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-0.11229 51.52269)
4,school,"The British School, Tehran","Devonshire House, 60 Goswell Rd, London, EC1M 7AD",1259,51.523182,-0.097971,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-0.09797 51.52318)


In [24]:
london_geodf = geoDataframe(london_df)

In [25]:
london_geodf.sample()

,query,name,address,distance,lat,lon,geometry
11,starbucks,Starbucks,"30-34A New Bridge St, London, Greater London, ...",385,51.512525,-0.103906,POINT (-0.10391 51.51252)


**Exporting dataset**

In [35]:
london_df.to_csv('./data/london_activities.csv', index = False)
london_geodf.to_csv('./data/london_geo_activities.csv', index = False)